**Función:** visualiza y confirma la etiqueta de cada fotos de un label en el dataset 10k (por limpiar).

**Requiere:**
- ../datasets/labels_to_review.json

**Crea:**
- ../datasets/final_rev_food.json
- ../datasets/final_rev_drink.json
- ../datasets/final_rev_outside.json
- ../datasets/final_rev_inside.json
- ../datasets/final_rev_menu.json

In [1]:
import os
import time
import json
from time import sleep
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from PIL import Image
from IPython.display import clear_output

In [ ]:
ejecutadoEnGoogleDrive = True # True si se ejecuta en Google Drive

In [2]:
if ejecutadoEnGoogleDrive:
    from google.colab import drive
    drive.mount('/content/drive')
    PATH = '/content/drive/clasificacion_imagenes'
else:
    PATH = '../'

Mounted at /content/drive


In [6]:
%%time
json_path = PATH + 'datasets/labels_to_review.json' # tiene menu (1397), outside (1189) y drink (1026)
df = pd.read_json(json_path, typ='frame')

CPU times: user 20.1 ms, sys: 1.94 ms, total: 22.1 ms
Wall time: 624 ms


In [7]:
df.head(2)

,photo_id,label
0,FBqe1mLbOdaGYgnw0nuELw,drink
1,L_nNWMC7nA8YlxMmu_tGsQ,drink


In [8]:
df.label.value_counts()

menu       1397
outside    1189
drink      1026
Name: label, dtype: int64

In [9]:
# Image Classifier
%matplotlib inline

label = ['food', 'inside', 'outside', 'drink', 'menu', 'KO']
range_ = list(range(1, 7))
label_dict = dict(zip(range_, label))
cat_dict = dict(zip(label, range_))

def check_labels(category, img_folder, check_folder, checkpoint_number, dataframe, start=None, end=None):
    while True:
      b = input('Pulsa 1 para empzar a clasificar la CATEGORIA: {}'.format(category))
      if b == '1':
        break
    ids_dict = {}

    if start == None and end == None:
        ids = list(dataframe[dataframe.label == category].photo_id)
    else:
        ids = list(dataframe[dataframe.label == category].photo_id)[start:end]

    len_ = len(ids)
    checkpoint = checkpoint_number

    # Loop por los photo_id de la categoría definida. Visualización y clasificación.
    i = 0
    while i < len(ids):
        current_label = list(dataframe[dataframe.photo_id == ids[i]].label)[0]

        # Control en el caso de que haya una imagen corrupta y no se pueda cargar. 
        while True:
            try:
                im = Image.open(img_folder + '/' + ids[i] + '.jpg')
                break

            except:
                ids_dict[ids[i]] = [current_label, 'Corrupt', 'KO']
                continue
        
        plt.title(f'{current_label}. (Quedan {len_} fotos)')
        plt.imshow(im)
        plt.show()

        print('¿Está bien tageada?')
        print('Intro: Sí,   1: food,   2: inside,   3: outside,   4: drink,   5: menu,   6: KO,   7: << volver')

        # Check si el valor está permitido y asignar las categorías.
        while True:

            # Guardar el diccionario en un json por seguridad
            if checkpoint == 0:
                json_name = f'checkpoint_{category}_v2.json'
                with open(check_folder + '/' + json_name, 'w') as save_check:
                    json.dump(ids_dict, save_check)
                
                print('Se ha grabado un checkpoint :)')
                checkpoint = checkpoint_number
                
            
            # Pedir opciones
            sleep(0.2)
            usr_input = input()
            
            if usr_input == '': # está bien etiquetada
                ids_dict[ids[i]] = [current_label, current_label, 'OK']
                checkpoint -= 1
                len_ -= 1
                i +=1 # avanzar
                break

            else:
                try:
                    if int(usr_input) == 7: # recular
                        i -= 1
                        len_ += 1
                        break
                    elif int(usr_input) == cat_dict[category]: # error. Misma categoria
                        print('Has tageado la misma categoría. Selecciona otra :)')
                        continue

                    elif int(usr_input) in list(label_dict.keys()): # reclasificada
                        ids_dict[ids[i]] = [current_label, label_dict[int(usr_input)], 'KO']
                        checkpoint -= 1
                        len_ -= 1
                        i +=1 # avanzar
                        break
                    else:
                        print('Valor equivocado :( Las opciones van del 1 al 6. Intenta otra vez.')
                        continue

                except:
                    print('Valor equivocado :( No es un número. Intenta otra vez.')

        clear_output(wait=True)
    
    json_name = f'final_rev_{category}.json'
    with open(check_folder + '/' + json_name, 'w') as save_check:
        json.dump(ids_dict, save_check)
    print('Se ha guardado toda la revisión de {} en {}'.format(category, json_name))

    return ids_dict

In [10]:
image_path = PATH + 'datasets/photos' # Ruta con las imagenes
save_path = PATH + 'datasets'           # Ruta donde queremos que se guade el json

label_a_revisar = ['food', 'drink','outside', 'inside','menu']

for label in label_a_revisar:
    check_labels(label,              # Categoria
                 image_path,         # ruta imagenes
                 save_path,          # ruta guardado checkpoint
                 20,                 # Frecuencia con la que guarda el checkpoint
                 df,                 # Dataframe de referencia                 
                 None,               # Dónde empieza el DF. Si está a None se carga entero (por defecto)
                 None)               # Dónde termona el DF. Start y end tiene que estar los dos a None o los dos con valor
print('Reclasificación completada. Gracias!')

Se ha guardado toda la revisión de drink en final_rev_drink.json
Reclasificación completada. Gracias!
